# Excluded Unknown Difficulty

In [1]:
import pandas as pd

# 输入与输出文件名
input_file = "full_equiv_output3_NoMismatch.xlsx"
output_file = "filtered_output.xlsx"

# 读取 Excel 文件
df = pd.read_excel(input_file)

# 检查是否存在 'Difficulty' 列
if 'Difficulty' not in df.columns:
    raise ValueError("Unknown Difficulty Column")

# 过滤掉 Difficulty 为 'UNKNOWN_DIFFICULTY' 的行
filtered_df = df[df['Difficulty'] != 'UNKNOWN_DIFFICULTY']

# 将结果写入新的 Excel 文件
filtered_df.to_excel(output_file, index=False)

print(f"Filter Succeed: {len(filtered_df)} lines of data saved to {output_file}")


Filter Succeed: 5037 lines of data saved to filtered_output.xlsx


# Gather Functional equivalent results from ChatGPT.

In [ ]:
import openai
import pandas as pd
import re
from openai import OpenAI

from dotenv import load_dotenv


# 加载 .env 文件
load_dotenv()

# 从环境变量读取 key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ========== User Configuration ==========
input_file = "full_equiv_output3_NoMismatch.xlsx" # This version of xlsx contains how many pass and fail solutions each problem has.
startline = 3001
endline = 3100
model_name = "gpt-4o"
output_file = f"full_equiv_output3_NoMismatch_{startline}_{endline}_YesNoOnly_{model_name}_tempEquals0.xlsx"

# ========================================

# --- Helper: Remove comments from C++, Java, and Python code ---
def remove_comments(code: str) -> str:
    # Remove /* ... */ block comments
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    # Remove // single-line comments
    code = re.sub(r"//.*", "", code)
    # Remove # single-line comments (Python)
    code = re.sub(r"#.*", "", code)
    # Remove Python triple-quoted strings (""" ... """ or ''' ... ''')
    code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
    code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
    return code.strip()

# Load Excel data
df = pd.read_excel(input_file)

# Define comparison pairs
pairs = [
    
    ("CPP_Pass1", "CPP_Pass2"),
    ("JAVA_Pass1", "JAVA_Pass2"),
    ("PYTHON3_Pass1", "PYTHON3_Pass2"),
    ("CPP_Pass1", "CPP_Fail"),
    ("JAVA_Pass1", "JAVA_Fail"),
    ("PYTHON3_Pass1", "PYTHON3_Fail"),
    ("CPP_Pass1", "JAVA_Pass1"),
    ("JAVA_Pass1", "PYTHON3_Pass1"),
    ("PYTHON3_Pass1", "CPP_Pass1"),
]

# Create result columns (only Yes/No)
for col1, col2 in pairs:
    df[f"{col1}_vs_{col2}"] = ""

# Process rows
for idx in range(startline - 1, min(endline, len(df))):
    row = df.iloc[idx]
    print(f"Processing row {idx + 1}...")

    problem = str(row.get("Description", "")).strip()

    for col1, col2 in pairs:
        code1 = str(row.get(col1, "")).strip()
        code2 = str(row.get(col2, "")).strip()

        if not code1 or not code2:
            continue

        # Remove comments
        clean_code1 = remove_comments(code1)
        clean_code2 = remove_comments(code2)

        # Prompt: only Yes/No
        prompt = f"""
You are a software analysis expert.

Determine if the following two code snippets (<code1> and <code2>) 
that aim to solve the problem (<Problem>) are functional equivalent. We define functional equivalence as two code snippets having the same input–output pairs. Ignore internal differences and focus only on whether the outputs are the same for identical inputs.

Reply with only one word: "Yes" or "No". 
Do not explain anything.

<Problem>
{problem}
</Problem>

<code1>
{clean_code1}
</code1>

<code2>
{clean_code2}
</code2>
"""
        # print(prompt)
        # Query GPT
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a software analysis expert."},
                    {"role": "user", "content": prompt}
                ],
                temperature = 0,
            )

            answer = response.choices[0].message.content.strip()
            # Normalize to "Yes" or "No"
            if answer.lower().startswith("y"):
                df.at[idx, f"{col1}_vs_{col2}"] = "Yes"
            elif answer.lower().startswith("n"):
                df.at[idx, f"{col1}_vs_{col2}"] = "No"
            else:
                df.at[idx, f"{col1}_vs_{col2}"] = "Unclear"

        except Exception as e:
            df.at[idx, f"{col1}_vs_{col2}"] = f"Error: {str(e)}"

# Save the result
df.to_excel(output_file, index=False)x
print(f"✅ Results saved to {output_file}")


Processing row 3001...
Processing row 3002...
Processing row 3003...
Processing row 3004...
Processing row 3005...
Processing row 3006...
Processing row 3007...
Processing row 3008...
Processing row 3009...
Processing row 3010...
Processing row 3011...
Processing row 3012...
Processing row 3013...
Processing row 3014...
Processing row 3015...
Processing row 3016...
Processing row 3017...
Processing row 3018...
Processing row 3019...
Processing row 3020...
Processing row 3021...
Processing row 3022...
Processing row 3023...
Processing row 3024...
Processing row 3025...
Processing row 3026...
Processing row 3027...
Processing row 3028...
Processing row 3029...
Processing row 3030...
Processing row 3031...
Processing row 3032...
Processing row 3033...
Processing row 3034...
Processing row 3035...
Processing row 3036...
Processing row 3037...
Processing row 3038...
Processing row 3039...
Processing row 3040...
Processing row 3041...
Processing row 3042...
Processing row 3043...
Processing 

# Calculate Column Proportion

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ========== 配置 ==========
input_file = "full_equiv_output3_NoMismatch_3001_3100_YesNoOnly_gpt-4o_tempEquals0.xlsx"
output_dir = "charts_output_3001_3100"
os.makedirs(output_dir, exist_ok=True)

columns_to_check = [
    "CPP_Pass1_vs_CPP_Pass2",
    "JAVA_Pass1_vs_JAVA_Pass2",
    "PYTHON3_Pass1_vs_PYTHON3_Pass2",
    "CPP_Pass1_vs_CPP_Fail",
    "JAVA_Pass1_vs_JAVA_Fail",
    "PYTHON3_Pass1_vs_PYTHON3_Fail",
    "CPP_Pass1_vs_JAVA_Pass1",
    "JAVA_Pass1_vs_PYTHON3_Pass1",
    "PYTHON3_Pass1_vs_CPP_Pass1"
]

difficulty_col = "Difficulty"
# =========================

df = pd.read_excel(input_file)

if difficulty_col not in df.columns:
    raise ValueError(f"❌ Column '{difficulty_col}' not found in Excel!")

df[difficulty_col] = df[difficulty_col].astype(str).str.strip()

total_yes = 0
total_no = 0
summary_data = []

# ---------- 创建大画布 (3×3) ----------
fig_overall, axs_overall = plt.subplots(3, 3, figsize=(15, 12))
fig_difficulty, axs_diff = plt.subplots(3, 3, figsize=(18, 14))

axs_overall = axs_overall.flatten()
axs_diff = axs_diff.flatten()

def compute_accuracy(col_name, yes_count, no_count):
    total = yes_count + no_count
    if total == 0:
        return 0
    if "Pass" in col_name and "Fail" in col_name:
        return no_count / total * 100  # No 是正确答案
    else:
        return yes_count / total * 100  # Yes 是正确答案

for i, col in enumerate(columns_to_check):
    if col not in df.columns:
        print(f"⚠️ Warning: Column '{col}' not found in Excel.")
        continue

    df[col] = df[col].astype(str).str.strip().str.lower()
    yes_count = (df[col] == "yes").sum()
    no_count = (df[col] == "no").sum()
    total_yes += yes_count
    total_no += no_count
    acc = compute_accuracy(col, yes_count, no_count)
    summary_data.append([col, yes_count, no_count, acc])

    # === Overall Yes/No ===
    ax = axs_overall[i]
    bars = ax.bar(["Yes", "No"], [yes_count, no_count], color=["green", "red"])
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height + 1, f"{height}", 
                ha='center', va='bottom', fontsize=8)
    ax.set_title(f"{col}\nAcc: {acc:.1f}%", fontsize=10)
    ax.set_ylabel("Count")
    ax.set_xlabel("")
    ax.grid(axis="y", linestyle="--", alpha=0.4)

    # === By Difficulty ===
    grouped = df.groupby(difficulty_col)[col].value_counts().unstack(fill_value=0)
    grouped = grouped.reindex(columns=["yes", "no"], fill_value=0)
    grouped.plot(
        kind="bar",
        stacked=True,
        color=["#4CAF50", "#F44336"],
        ax=axs_diff[i],
        legend=False
    )

    # 计算每个难度的正确率并在柱上标注
    for idx, (diff, row) in enumerate(grouped.iterrows()):
        y, n = row.get("yes", 0), row.get("no", 0)
        acc_diff = compute_accuracy(col, y, n)
        axs_diff[i].text(idx, y + n + 0.5, f"{acc_diff:.1f}%", ha="center", fontsize=8, color="blue")

    axs_diff[i].set_title(f"{col}\nAcc by Diff", fontsize=10)
    axs_diff[i].set_xlabel("Difficulty")
    axs_diff[i].set_ylabel("Count")
    axs_diff[i].grid(axis="y", linestyle="--", alpha=0.3)

# ---------- 调整布局并保存 ----------
for fig in [fig_overall, fig_difficulty]:
    fig.tight_layout(pad=3.0)

fig_overall.suptitle("Overall Yes/No Counts (with Accuracy%)", fontsize=16, y=1.02)
fig_difficulty.suptitle("Yes/No by Difficulty (with Accuracy%)", fontsize=16, y=1.02)

fig_overall.savefig(os.path.join(output_dir, "ALL_overall_with_accuracy.png"), dpi=300, bbox_inches="tight")
fig_difficulty.savefig(os.path.join(output_dir, "ALL_by_difficulty_with_accuracy.png"), dpi=300, bbox_inches="tight")

plt.close(fig_overall)
plt.close(fig_difficulty)

# === 总汇总 ===
summary_df = pd.DataFrame(summary_data, columns=["Column", "Yes", "No", "Accuracy"])
summary_df.plot(
    x="Column",
    y=["Yes", "No"],
    kind="bar",
    stacked=True,
    color=["#4CAF50", "#F44336"],
    figsize=(10, 6)
)
plt.title("Overall Yes/No Count per Column")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "overall_summary_single.png"), dpi=300)
plt.close()

# === 输出控制台统计 ===
print("\n===========================")
print("📈 All combined charts saved to:", os.path.abspath(output_dir))
print(f"  ✅ Total Yes: {total_yes}")
print(f"  ❌ Total No:  {total_no}")
print("\n📊 Accuracy Summary:")
print(summary_df.to_string(index=False, float_format="%.2f"))



📈 All combined charts saved to: c:\Users\Hui Sun\Dropbox\Code Readbility LLM\ComparativeComprehensionLLM\charts_output_3001_3100
  ✅ Total Yes: 444
  ❌ Total No:  456

📊 Accuracy Summary:
                        Column  Yes  No  Accuracy
        CPP_Pass1_vs_CPP_Pass2   67  33     67.00
      JAVA_Pass1_vs_JAVA_Pass2   69  31     69.00
PYTHON3_Pass1_vs_PYTHON3_Pass2   60  40     60.00
         CPP_Pass1_vs_CPP_Fail   28  72     72.00
       JAVA_Pass1_vs_JAVA_Fail   39  61     61.00
 PYTHON3_Pass1_vs_PYTHON3_Fail   22  78     78.00
       CPP_Pass1_vs_JAVA_Pass1   63  37     63.00
   JAVA_Pass1_vs_PYTHON3_Pass1   55  45     55.00
    PYTHON3_Pass1_vs_CPP_Pass1   41  59     41.00


# GPT-5 Result

In [ ]:
import time
import re
import pandas as pd
from openai import OpenAI
import traceback

# ===============================
# Initialize OpenAI client
# ===============================
client = OpenAI(api_key="")

# ========== User Configuration ==========
input_file = "full_equiv_output3_NoMismatch.xlsx"
startline = 1
endline = 3
output_file = f"full_equiv_output3_NoMismatch_{startline}_{endline}_YesNoOnly_GPT5_log.xlsx"
model_name = "gpt-5-mini"  
# ========================================

# --- Helper: Remove comments ---
def remove_comments(code: str) -> str:
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    code = re.sub(r"//.*", "", code)
    code = re.sub(r"#.*", "", code)
    code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
    code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
    return code.strip()

# --- Load Excel ---
print("📖 Loading Excel file...")
df = pd.read_excel(input_file)
print(f"✅ Loaded {len(df)} rows from {input_file}")

# --- Define pairs ---
pairs = [
    ("CPP_Pass1", "CPP_Pass2"),
    ("JAVA_Pass1", "JAVA_Pass2"),
    ("PYTHON3_Pass1", "PYTHON3_Pass2"),
    ("CPP_Pass1", "CPP_Fail"),
    ("JAVA_Pass1", "JAVA_Fail"),
    ("PYTHON3_Pass1", "PYTHON3_Fail"),
    ("CPP_Pass1", "JAVA_Pass1"),
    ("JAVA_Pass1", "PYTHON3_Pass1"),
    ("PYTHON3_Pass1", "CPP_Pass1"),
]

# --- Create result columns ---
for col1, col2 in pairs:
    df[f"{col1}_vs_{col2}"] = ""

# --- Main loop ---
for idx in range(startline - 1, min(endline, len(df))):
    row = df.iloc[idx]
    print(f"\n🚀 [ROW {idx + 1}] Start processing...")
    t0 = time.time()

    problem = str(row.get("Description", "")).strip()
    print(f"🧩 Problem name: {problem[:80]}{'...' if len(problem) > 80 else ''}")

    for col1, col2 in pairs:
        out_col = f"{col1}_vs_{col2}"
        code1 = str(row.get(col1, "")).strip()
        code2 = str(row.get(col2, "")).strip()

        if not code1 or not code2:
            print(f"⚠️  {col1} or {col2} missing — skipping")
            df.at[idx, out_col] = "Unclear"
            continue

        clean_code1 = remove_comments(code1)
        clean_code2 = remove_comments(code2)

        print(f"🧠 Preparing prompt for {col1} vs {col2} ...")

        prompt = f"""
You are a software analysis expert.

Determine if the following two code snippets (<code1> and <code2>) 
that aim to solve the problem (<Problem>) are functionally equivalent.
Definition: they produce the SAME input-output mapping for all valid inputs of the problem.
Ignore inner implementation differences; only focus on input/output pairs.

Reply with only one word: "Yes" or "No".
Do not explain anything.

<Problem>
{problem}
</Problem>

<code1>
{clean_code1}
</code1>

<code2>
{clean_code2}
</code2>
"""

        try:
            print(f"📤 Sending request to ChatGPT ({model_name})...")
            start_api = time.time()
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a software analysis expert."},
                    {"role": "user", "content": prompt}
                ],
                timeout=120  # 最长等 2 分钟
            )
            print(f"📥 Response received in {round(time.time() - start_api, 2)}s")

            answer = response.choices[0].message.content.strip()
            print(f"💬 GPT answer: {answer[:50]}{'...' if len(answer) > 50 else ''}")

            ans_lower = answer.lower()
            if ans_lower.startswith("y"):
                df.at[idx, out_col] = "Yes"
            elif ans_lower.startswith("n"):
                df.at[idx, out_col] = "No"
            else:
                df.at[idx, out_col] = "Unclear"

        except Exception as e:
            print(f"❌ ERROR during {col1} vs {col2}: {type(e).__name__}")
            traceback.print_exc()
            df.at[idx, out_col] = f"Error: {type(e).__name__}: {str(e)}"

    print(f"✅ [ROW {idx + 1}] Finished in {round(time.time() - t0, 2)}s")

# --- Save output ---
df.to_excel(output_file, index=False)
print(f"\n🎉 All done! Results saved to {output_file}")


📖 Loading Excel file...
✅ Loaded 6905 rows from full_equiv_output3_NoMismatch.xlsx

🚀 [ROW 1] Start processing...
🧩 Problem name: 1000_A. Codehorses T-shirts
🧠 Preparing prompt for CPP_Pass1 vs CPP_Pass2 ...
📤 Sending request to ChatGPT (gpt-5-mini)...
📥 Response received in 110.18s
💬 GPT answer: No
🧠 Preparing prompt for JAVA_Pass1 vs JAVA_Pass2 ...
📤 Sending request to ChatGPT (gpt-5-mini)...
📥 Response received in 21.68s
💬 GPT answer: No
🧠 Preparing prompt for PYTHON3_Pass1 vs PYTHON3_Pass2 ...
📤 Sending request to ChatGPT (gpt-5-mini)...
📥 Response received in 15.92s
💬 GPT answer: Yes
🧠 Preparing prompt for CPP_Pass1 vs CPP_Fail ...
📤 Sending request to ChatGPT (gpt-5-mini)...
📥 Response received in 40.13s
💬 GPT answer: No
🧠 Preparing prompt for JAVA_Pass1 vs JAVA_Fail ...
📤 Sending request to ChatGPT (gpt-5-mini)...
📥 Response received in 18.82s
💬 GPT answer: No
🧠 Preparing prompt for PYTHON3_Pass1 vs PYTHON3_Fail ...
📤 Sending request to ChatGPT (gpt-5-mini)...
📥 Response receive

# Save code into folder


In [ ]:
import os
import pandas as pd
import re


input_file = "full_equiv_output3_NoMismatch_1_101_YesNoOnly_gpt-4o_tempEquals0.xlsx"
start_line = 1   
end_line = 101   
output_base = "Code"



os.makedirs(output_base, exist_ok=True)


df = pd.read_excel(input_file)


columns_to_export = [
    "CPP_Pass1", "CPP_Pass2", "CPP_Fail",
    "JAVA_Pass1", "JAVA_Pass2", "JAVA_Fail",
    "PYTHON3_Pass1", "PYTHON3_Pass2", "PYTHON3_Fail"
]


columns_to_export = [c for c in columns_to_export if c in df.columns]

for idx in range(start_line - 1, min(end_line, len(df))):
    row = df.iloc[idx]
    problem_name = str(row.get("ProblemName", f"Row_{idx+1}"))
    

    folder_name = re.sub(r'[<>:"/\\|?*]', '_', problem_name)
    folder_path = os.path.join(output_base, folder_name)
    os.makedirs(folder_path, exist_ok=True)


    for col in columns_to_export:
        value = str(row.get(col, "")).strip()
        file_path = os.path.join(folder_path, f"{col}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(value)

print(f"✅ Saved to：{os.path.abspath(output_base)}")


✅ 数据已导出至：c:\Users\Hui Sun\Dropbox\Code Readbility LLM\ComparativeComprehensionLLM\Code


# QWen


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import re
import gc

# ========== User Configuration ==========
input_file = "filtered_output.xlsx"
startline = 1
endline = 3
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
output_file = f"Qwen/full_equiv_output3_NoMismatch_{startline}_{endline}_YesNoOnly_Qwen.xlsx"
device = "cuda" if torch.cuda.is_available() else "cpu"
# ========================================

# --- Helper: Remove comments ---
def remove_comments(code: str) -> str:
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    code = re.sub(r"//.*", "", code)
    code = re.sub(r"#.*", "", code)
    code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
    code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
    return code.strip()

# --- Load model ---
print(f"Loading model {model_name} ...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)
model.eval()
print("Model loaded successfully.")

# --- Load Excel data ---
df = pd.read_excel(input_file)

# --- Define code pairs for comparison ---
pairs = [
    ("CPP_Pass1", "CPP_Pass2"),
    ("JAVA_Pass1", "JAVA_Pass2"),
    ("PYTHON3_Pass1", "PYTHON3_Pass2"),
    ("CPP_Pass1", "CPP_Fail"),
    ("JAVA_Pass1", "JAVA_Fail"),
    ("PYTHON3_Pass1", "PYTHON3_Fail"),
    ("CPP_Pass1", "JAVA_Pass1"),
    ("JAVA_Pass1", "PYTHON3_Pass1"),
    ("PYTHON3_Pass1", "CPP_Pass1"),
]

# --- Create empty result columns ---
for col1, col2 in pairs:
    df[f"{col1}_vs_{col2}"] = ""

# --- Process each row ---
for idx in range(startline - 1, min(endline, len(df))):
    print(f"Processing row {idx + 1}...")
    row = df.iloc[idx]
    problem = str(row.get("Description", "")).strip()

    for col1, col2 in pairs:
        code1 = str(row.get(col1, "")).strip()
        code2 = str(row.get(col2, "")).strip()
        if not code1 or not code2:
            continue

        clean_code1 = remove_comments(code1)
        clean_code2 = remove_comments(code2)

        prompt = f"""
Determine if the two code snippets are functionally equivalent according to the problem.


<problem>
{problem}
</problem>
<code1>
{clean_code1}
</code1>
<code2>
{clean_code2}
</code2>
Reply with only Yes or No.
"""

        try:
            with torch.no_grad():
                inputs = tokenizer(prompt, return_tensors="pt").to(device)
                outputs = model.generate(**inputs, max_new_tokens=20)
                answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

            # Normalize to Yes/No
            if "yes" in answer.lower():
                df.at[idx, f"{col1}_vs_{col2}"] = "Yes"
            elif "no" in answer.lower():
                df.at[idx, f"{col1}_vs_{col2}"] = "No"
            else:
                df.at[idx, f"{col1}_vs_{col2}"] = "Unclear"

        except torch.cuda.OutOfMemoryError:
            print("⚠️ CUDA out of memory — clearing cache and continuing...")
            torch.cuda.empty_cache()
            df.at[idx, f"{col1}_vs_{col2}"] = "Error: CUDA OOM"
        except Exception as e:
            df.at[idx, f"{col1}_vs_{col2}"] = f"Error: {str(e)}"

        # ✅ Free GPU memory every iteration
        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()

# --- Save the results ---
df.to_excel(output_file, index=False)
print(f"✅ Results saved to {output_file}")


Loading model Qwen/Qwen2.5-Coder-7B-Instruct ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
import os
import gc
import re
import torch
import pandas as pd
from functools import lru_cache
from transformers import AutoTokenizer, AutoModelForCausalLM

# ========== User Configuration ==========
input_file = "filtered_output.xlsx"
startline = 1
endline = 3
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
output_file = f"Qwen/full_equiv_output3_NoMismatch_{startline}_{endline}_YesNoOnly_Qwen_light.xlsx"
autosave_every = 10  # 每处理N行自动保存一次
# 可选：限制单个样本的最大长度；None 表示不截断
MAX_LENGTH = None      # e.g., 4096
TRUNCATION = False if MAX_LENGTH is None else True
# ========================================

# ========== 环境与进度条 ==========
device = "cuda" if torch.cuda.is_available() else "cpu"
try:
    from tqdm.notebook import tqdm as _tqdm
except Exception:
    try:
        from tqdm import tqdm as _tqdm
    except Exception:
        def _tqdm(x, **kwargs):
            return x

print(f"[Info] Device detected: {device}")

# ---------- 去注释 ----------
def remove_comments(code: str) -> str:
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)   # C/Java block
    code = re.sub(r"//.*", "", code)                         # C/Java line
    code = re.sub(r"#.*", "", code)                          # Python line
    code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)   # Python triple
    code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)   # Python triple
    return code.strip()

# ---------- 加载模型（含CPU安全回退） ----------
print(f"[Info] Loading model {model_name} ...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = None
if device == "cuda":
    # 优先尝试 8-bit（显存友好）
    try:
        from transformers import BitsAndBytesConfig
        bnb_config = BitsAndBytesConfig(load_in_8bit=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="cuda",
            quantization_config=bnb_config,
        )
        print("[Info] Model loaded in 8-bit on CUDA.")
    except Exception as e:
        print(f"[Warn] 8-bit load failed ({e}). Falling back to fp16 on CUDA...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="cuda",
        )
else:
    # 无GPU：CPU + fp32（会慢，但稳定）
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,
        device_map=None,
        low_cpu_mem_usage=True
    )
def first_token_id(s: str):
    ids = tokenizer(s, add_special_tokens=False).input_ids
    return ids[0] if len(ids) > 0 else None

YES_CANDIDATES = [" Yes", "Yes", " yes"]
NO_CANDIDATES  = [" No", "No", " no"]

yes_ids = [tid for s in YES_CANDIDATES for tid in [first_token_id(s)] if tid is not None]
no_ids  = [tid for s in NO_CANDIDATES  for tid in [first_token_id(s)] if tid is not None]

if not yes_ids or not no_ids:
    raise RuntimeError("Tokenizer could not produce token ids for Yes/No. Check tokenizer and candidates.")

# ---------- Prompt 生成 ----------
def make_prompt(problem: str, code1: str, code2: str) -> str:
    return (
        "Determine if the two code snippets are functionally equivalent according to the problem.\n\n"
        "<problem>\n" + problem + "\n</problem>\n"
        "<code1>\n" + code1 + "\n</code1>\n"
        "<code2>\n" + code2 + "\n</code2>\n"
        "Reply with only Yes or No."
    )

# tokenizer 缓存（避免重复编码相同文本）
@lru_cache(maxsize=10000)
def encode_text_cached_single(text: str, truncation: bool, max_length):
    return tokenizer(
        text,
        return_tensors="pt",
        truncation=truncation,
        max_length=max_length
    )

# ---------- 单个 prompt 前向推理，返回 "Yes"/"No" 或错误 ----------
def infer_single(prompt: str) -> str:
    input_i = None
    output_i = None
    logits = None
    probs = None
    try:
        # 编码
        input_i = encode_text_cached_single(
            prompt,
            truncation=TRUNCATION,
            max_length=MAX_LENGTH
        )

        # 上设备
        if device == "cuda":
            input_i = {k: v.to("cuda") for k, v in input_i.items()}

        # 前向
        output_i = model(**input_i)           # logits: [1, T, V]
        logits = output_i.logits[:, -1, :]    # [1, V]
        probs = torch.softmax(logits, dim=-1) # [1, V]

        yes_prob = probs[:, yes_ids].sum().item()
        no_prob  = probs[:,  no_ids].sum().item()
        return "Yes" if yes_prob > no_prob else "No"

    except torch.cuda.OutOfMemoryError:
        # 显存溢出：清理并返回错误
        if device == "cuda":
            print("[OOM] CUDA OOM on a single prompt, recovering...")
            torch.cuda.empty_cache()
        return "Error: CUDA OOM"
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        # 安全清理，避免 NameError
        for var_name in ["input_i", "output_i", "logits", "probs"]:
            if var_name in locals():
                try:
                    del locals()[var_name]
                except Exception:
                    pass
        gc.collect()
        if device == "cuda":
            # 同步+清空缓存，更干净
            try:
                torch.cuda.synchronize()
            except Exception:
                pass
            torch.cuda.empty_cache()

# ---------- 主循环：逐行、逐pair 逐个推理 ----------
row_range = range(startline - 1, min(endline, len(df)))
for ridx in _tqdm(row_range, desc="Rows"):
    row = df.iloc[ridx]
    problem = str(row.get("Description", "")).strip()

    # 本行有效 pair
    pair_items = []
    for col1, col2 in pairs:
        code1 = str(row.get(col1, "")).strip()
        code2 = str(row.get(col2, "")).strip()
        colname = f"{col1}_vs_{col2}"
        if not code1 or not code2:
            continue

        clean_code1 = remove_comments(code1)
        clean_code2 = remove_comments(code2)
        prompt = make_prompt(problem, clean_code1, clean_code2)
        pair_items.append((colname, prompt))

    if not pair_items:
        continue

    # 逐个推理，避免一次性大批量占显存
    for colname, prompt in pair_items:
        result = infer_single(prompt)
        df.at[ridx, colname] = result

    # 自动保存进度
    if ((ridx + 1) % autosave_every) == 0 or (ridx == row_range.stop - 1):
        try:
            df.to_excel(output_file, index=False)
            print(f"[Autosave] Progress saved at row {ridx + 1} -> {output_file}")
        except Exception as e:
            print(f"[Warn] Autosave failed at row {ridx + 1}: {e}")

# 最终保存
df.to_excel(output_file, index=False)
print(f"✅ Done. Results saved to {output_file}")

[Info] Device detected: cuda
[Info] Loading model Qwen/Qwen2.5-Coder-7B-Instruct ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[Info] Model loaded in 8-bit on CUDA.


Rows:   0%|          | 0/3 [00:00<?, ?it/s]

[Autosave] Progress saved at row 3 -> Qwen/full_equiv_output3_NoMismatch_1_3_YesNoOnly_Qwen_light.xlsx
✅ Done. Results saved to Qwen/full_equiv_output3_NoMismatch_1_3_YesNoOnly_Qwen_light.xlsx


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())

True
12.8
91002


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 模型名称
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

# 检查设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载模型（半精度 + 自动分配设备）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)
model.eval()

# 测试 Prompt
prompt = """
Determine if the two code snippets are functionally equivalent according to the problem.
<problem>
During the research on properties of the greatest common divisor (GCD) of a set of numbers, Ildar, a famous mathematician, introduced a brand new concept of the weakened common divisor (WCD) of a list of pairs of integers.

For a given list of pairs of integers (a_1, b_1), (a_2, b_2), ..., (a_n, b_n) their WCD is arbitrary integer greater than 1, such that it divides at least one element in each pair. WCD may not exist for some lists.

For example, if the list looks like [(12, 15), (25, 18), (10, 24)], then their WCD can be equal to 2, 3, 5 or 6 (each of these numbers is strictly greater than 1 and divides at least one number in each pair).

You're currently pursuing your PhD degree under Ildar's mentorship, and that's why this problem was delegated to you. Your task is to calculate WCD efficiently.

Input

The first line contains a single integer n (1 ≤ n ≤ 150 000) — the number of pairs.

Each of the next n lines contains two integer values a_i, b_i (2 ≤ a_i, b_i ≤ 2 ⋅ 10^9).

Output

Print a single integer — the WCD of the set of pairs. 

If there are multiple possible answers, output any; if there is no answer, print -1.

Examples

Input

3
17 18
15 24
12 15


Output

6

Input

2
10 16
7 17


Output

-1


Input

5
90 108
45 105
75 40
165 175
33 30


Output

5

Note

In the first example the answer is 6 since it divides 18 from the first pair, 24 from the second and 12 from the third ones. Note that other valid answers will also be accepted.

In the second example there are no integers greater than 1 satisfying the conditions.

In the third example one of the possible answers is 5. Note that, for example, 15 is also allowed, but it's not necessary to maximize the output.
</problem>
<code1>
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStream;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.util.ArrayList;
import java.util.StringTokenizer;

public class B {
	public static void main(String[] args) throws IOException {
		FastScanner in= new FastScanner(System.in);
		PrintWriter out= new PrintWriter(System.out);
		int n= in.nextInt();
		long [] a= new long[n];
		long [] b= new long[n];
		long g= 0L;
		for (int i = 0; i < b.length; i++) {
			a[i]= in.nextLong();
			b[i]= in.nextLong();
			g= gcd(g, a[i]*b[i]); 
		}
		if(g==1) {
			System.out.println(-1);
			return;
		}
		for (int i = 0; i < b.length; i++) {
			long thing= gcd(g, a[i]);
			if(thing!=1) g= thing;
			thing= gcd(g, b[i]);
			if(thing!=1) g= thing;
		}
		System.out.println(g);

	}
	public static long gcd(long a, long b) {
		return b==0 ? a: gcd(b, a%b);
	}
	static class p{
		int id;
		int par;
		public p(int a, int b) {
			id= a;
			par= b;
		}
	}
	static class FenwickTree {
		long[] ft;
		public FenwickTree(int n) {
			ft = new long[n + 1];
		}

		int rsq(int b) {
			int sum = 0;
			for (; b > 0; b -= (b & (-b)))
				sum += ft[b];
			return sum;
		}

		int rsq(int a, int b) {
			return rsq(b) - (a == 1 ? 0 : rsq(a - 1));
		}

		void update(int k, int v) {
			for (; k < ft.length; k += (k & (-k)))
				ft[k] += v;
		}
	}

	static class FastScanner {
		BufferedReader br;
		StringTokenizer st;

		public FastScanner(InputStream in) {
			br = new BufferedReader(new InputStreamReader(in));
			st = new StringTokenizer("");
		}

		public String next() throws IOException {
			if (!st.hasMoreTokens()) {
				st = new StringTokenizer(br.readLine());
				return next();
			}
			return st.nextToken();
		}

		public int nextInt() throws IOException {
			return Integer.parseInt(next());
		}
		public double nextDouble() throws NumberFormatException, IOException {
			return Double.parseDouble(next());
		}
		public long nextLong() throws IOException {
			return Long.parseLong(next());
		}
	}
}
</code1>
<code2>
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.util.StringTokenizer;

public class Prime1025B {
	// -------------------
	public static final int MOD = (int) (1e9 + 7);
	// ------------------

	BufferedReader br = new BufferedReader(new InputStreamReader(System.in));
	StringTokenizer st;
	PrintWriter out = new PrintWriter(System.out);

	String next() {
		while (st == null || !st.hasMoreElements()) {
			try {
				st = new StringTokenizer(br.readLine());
			} catch (IOException e) {
				e.printStackTrace();
			}
		}
		return st.nextToken();
	}

	int nextInt() {
		return Integer.parseInt(next());
	}

	long nextLong() {
		return Long.parseLong(next());
	}

	// ------------------
	public static void main(String[] args) {
		Prime1025B ob = new Prime1025B();
		ob.run();
	}

	private void run() {
		
		int n = nextInt();
		long[] a = new long[n];
		long[] b = new long[n];
		long g =0;
		for(int i=0;i<n;i++) {
			a[i] = nextInt(); b[i] = nextInt();
			g = gcd(g, a[i]*b[i]);
		}
		if(g==1L) {
			out.println("-1");
		}else {
//			out.println(g);
			long temp=0;
			for(int i=0;i<n;i++) {
				temp = gcd(g, a[i]);
				if(temp==1) {
					temp = gcd(g, b[i]);
				}
				g = temp;
			}
			out.println(g);
		}
		out.close();
	}
	private long gcd(long a, long b) {
		while(b!=0) {
			long t = a%b;
			a = b;
			b = t;
		}
		return a;
	}

}
</code2>
Reply with only Yes or No. 
"""

# 编码并生成
with torch.inference_mode():
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 打印结果
print("\n===============================")
print("Prompt:")
print(prompt.strip())
print("===============================")
print("Model Output:")
print(result.strip())
print("===============================")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.



Prompt:
Determine if the two code snippets are functionally equivalent according to the problem.
<problem>
During the research on properties of the greatest common divisor (GCD) of a set of numbers, Ildar, a famous mathematician, introduced a brand new concept of the weakened common divisor (WCD) of a list of pairs of integers.

For a given list of pairs of integers (a_1, b_1), (a_2, b_2), ..., (a_n, b_n) their WCD is arbitrary integer greater than 1, such that it divides at least one element in each pair. WCD may not exist for some lists.

For example, if the list looks like [(12, 15), (25, 18), (10, 24)], then their WCD can be equal to 2, 3, 5 or 6 (each of these numbers is strictly greater than 1 and divides at least one number in each pair).

You're currently pursuing your PhD degree under Ildar's mentorship, and that's why this problem was delegated to you. Your task is to calculate WCD efficiently.

Input

The first line contains a single integer n (1 ≤ n ≤ 150 000) — the number

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 模型名称
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

# 检查设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载模型（半精度 + 自动分配设备）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)
model.eval()

# 测试 Prompt
prompt = """
Determine if the two code snippets are functionally equivalent according to the problem.
<problem>
During the research on properties of the greatest common divisor (GCD) of a set of numbers, Ildar, a famous mathematician, introduced a brand new concept of the weakened common divisor (WCD) of a list of pairs of integers.

For a given list of pairs of integers (a_1, b_1), (a_2, b_2), ..., (a_n, b_n) their WCD is arbitrary integer greater than 1, such that it divides at least one element in each pair. WCD may not exist for some lists.

For example, if the list looks like [(12, 15), (25, 18), (10, 24)], then their WCD can be equal to 2, 3, 5 or 6 (each of these numbers is strictly greater than 1 and divides at least one number in each pair).

You're currently pursuing your PhD degree under Ildar's mentorship, and that's why this problem was delegated to you. Your task is to calculate WCD efficiently.

Input

The first line contains a single integer n (1 ≤ n ≤ 150 000) — the number of pairs.

Each of the next n lines contains two integer values a_i, b_i (2 ≤ a_i, b_i ≤ 2 ⋅ 10^9).

Output

Print a single integer — the WCD of the set of pairs. 

If there are multiple possible answers, output any; if there is no answer, print -1.

Examples

Input

3
17 18
15 24
12 15


Output

6

Input

2
10 16
7 17


Output

-1


Input

5
90 108
45 105
75 40
165 175
33 30


Output

5

Note

In the first example the answer is 6 since it divides 18 from the first pair, 24 from the second and 12 from the third ones. Note that other valid answers will also be accepted.

In the second example there are no integers greater than 1 satisfying the conditions.

In the third example one of the possible answers is 5. Note that, for example, 15 is also allowed, but it's not necessary to maximize the output.
</problem>
<code1>
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStream;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.util.ArrayList;
import java.util.StringTokenizer;

public class B {
	public static void main(String[] args) throws IOException {
		FastScanner in= new FastScanner(System.in);
		PrintWriter out= new PrintWriter(System.out);
		int n= in.nextInt();
		long [] a= new long[n];
		long [] b= new long[n];
		long g= 0L;
		for (int i = 0; i < b.length; i++) {
			a[i]= in.nextLong();
			b[i]= in.nextLong();
			g= gcd(g, a[i]*b[i]); 
		}
		if(g==1) {
			System.out.println(-1);
			return;
		}
		for (int i = 0; i < b.length; i++) {
			long thing= gcd(g, a[i]);
			if(thing!=1) g= thing;
			thing= gcd(g, b[i]);
			if(thing!=1) g= thing;
		}
		System.out.println(g);

	}
	public static long gcd(long a, long b) {
		return b==0 ? a: gcd(b, a%b);
	}
	static class p{
		int id;
		int par;
		public p(int a, int b) {
			id= a;
			par= b;
		}
	}
	static class FenwickTree {
		long[] ft;
		public FenwickTree(int n) {
			ft = new long[n + 1];
		}

		int rsq(int b) {
			int sum = 0;
			for (; b > 0; b -= (b & (-b)))
				sum += ft[b];
			return sum;
		}

		int rsq(int a, int b) {
			return rsq(b) - (a == 1 ? 0 : rsq(a - 1));
		}

		void update(int k, int v) {
			for (; k < ft.length; k += (k & (-k)))
				ft[k] += v;
		}
	}

	static class FastScanner {
		BufferedReader br;
		StringTokenizer st;

		public FastScanner(InputStream in) {
			br = new BufferedReader(new InputStreamReader(in));
			st = new StringTokenizer("");
		}

		public String next() throws IOException {
			if (!st.hasMoreTokens()) {
				st = new StringTokenizer(br.readLine());
				return next();
			}
			return st.nextToken();
		}

		public int nextInt() throws IOException {
			return Integer.parseInt(next());
		}
		public double nextDouble() throws NumberFormatException, IOException {
			return Double.parseDouble(next());
		}
		public long nextLong() throws IOException {
			return Long.parseLong(next());
		}
	}
}
</code1>
<code2>
import java.io.*;
import java.util.*;
import java.math.*;
import java.lang.*;
 
import static java.lang.Math.*;
 
public class wgcd implements Runnable
{
    static class InputReader
    {
        private InputStream stream;
        private byte[] buf = new byte[1024];
        private int curChar;
        private int numChars;
        private SpaceCharFilter filter;
        private BufferedReader br=new BufferedReader(new InputStreamReader(System.in));
 
        public InputReader(InputStream stream)
        {
            this.stream = stream;
        }
        
        public int read()
        {
            if (numChars==-1) 
                throw new InputMismatchException();
            
            if (curChar >= numChars)
            {
                curChar = 0;
                try 
                {
                    numChars = stream.read(buf);
                }
                catch (IOException e)
                {
                    throw new InputMismatchException();
                }
                
                if(numChars <= 0)               
                    return -1;
            }
            return buf[curChar++];
        }
     
        public String nextLine()
        {
            String str = "";
            try
            {
                str = br.readLine();
            }
            catch (IOException e)
            {
                e.printStackTrace();
            }
            return str;
        }
        public int nextInt()
        {
            int c = read();
            
            while(isSpaceChar(c)) 
                c = read();
            
            int sgn = 1;
            
            if (c == '-') 
            {
                sgn = -1;
                c = read();
            }
            
            int res = 0;
            do 
            {
                if(c<'0'||c>'9') 
                    throw new InputMismatchException();
                res *= 10;
                res += c - '0';
                c = read();
            }
            while (!isSpaceChar(c)); 
            
            return res * sgn;
        }
        
        public long nextLong() 
        {
            int c = read();
            while (isSpaceChar(c))
                c = read();
            int sgn = 1;
            if (c == '-') 
            {
                sgn = -1;
                c = read();
            }
            long res = 0;
            
            do 
            {
                if (c < '0' || c > '9')
                    throw new InputMismatchException();
                res *= 10;
                res += c - '0';
                c = read();
            }
            while (!isSpaceChar(c));
                return res * sgn;
        }
        
        public double nextDouble() 
        {
            int c = read();
            while (isSpaceChar(c))
                c = read();
            int sgn = 1;
            if (c == '-') 
            {
                sgn = -1;
                c = read();
            }
            double res = 0;
            while (!isSpaceChar(c) && c != '.') 
            {
                if (c == 'e' || c == 'E')
                    return res * Math.pow(10, nextInt());
                if (c < '0' || c > '9')
                    throw new InputMismatchException();
                res *= 10;
                res += c - '0';
                c = read();
            }
            if (c == '.') 
            {
                c = read();
                double m = 1;
                while (!isSpaceChar(c)) 
                {
                    if (c == 'e' || c == 'E')
                        return res * Math.pow(10, nextInt());
                    if (c < '0' || c > '9')
                        throw new InputMismatchException();
                    m /= 10;
                    res += (c - '0') * m;
                    c = read();
                }
            }
            return res * sgn;
        }
        
        public String readString() 
        {
            int c = read();
            while (isSpaceChar(c))
                c = read();
            StringBuilder res = new StringBuilder();
            do 
            {
                res.appendCodePoint(c);
                c = read();
            } 
            while (!isSpaceChar(c));
            
            return res.toString();
        }
     
        public boolean isSpaceChar(int c) 
        {
            if (filter != null)
                return filter.isSpaceChar(c);
            return c == ' ' || c == '\n' || c == '\r' || c == '\t' || c == -1;
        }
     
        public String next() 
        {
            return readString();
        }
        
        public interface SpaceCharFilter 
        {
            public boolean isSpaceChar(int ch);
        }
    }
    
    public static void main(String args[]) throws Exception
    {
        new Thread(null, new wgcd(),"wgcd",1<<26).start();
    }
    /*class Wand {
        int ind;
        int val;
        Wand(int a, int b) {
            ind = a;
            val = b;
        }
    }*/ //(This statement shows how to make our own class)
    
    //to count no of 1 in binary form::
    long count_one (int n){
            long count=0;
            while(n>0)
            {
               n = n&(n-1);
               count++;
            }
            return count;
       }
    //To Find most significant setbit of a number::
    int Last_set_bit(int n){
        return (int)(Math.log(n)/Math.log(2));
    }   
    //power mod::
    static long powmod(long x, long n, long m) {
        if (n == 0) return 1;

        else if (n % 2 == 0) return powmod(((x % m) * (x % m)) % m, n / 2, m);

        else if (n % 2 == 1) return (((x % m) * powmod(((x % m) * (x % m)) % m, (n - 1) / 2, m)) % m);

        else return 0;
    }
    //binary expo::
    static long binaryexpo(long x, long n){
        if(n==0) return 1;
        if(n%2==0) return binaryexpo(x*x,n);
        else return x*binaryexpo(x*x,(n-1)%2);
    }
    //GCD od 2 nos::
    public static long gcd(long a, long b) {
        if(a==0) return b;
        return gcd(b%a,a);
    }
    //GCD of Array::
    public static long gcd(long[] input) {
        long result = input[0];
 
        for (int i = 1; i < input.length; i++) {
            result = gcd(result, input[i]);
        }
 
        return result;
    }
    //LCM of 2 nos::
    public static long lcm(long a, long b) {
        return a * (b / gcd(a, b));
    }
    //LCM of Array::
    public static long lcm(long[] input) {
        long result = input[0];
 
        for (int i = 1; i < input.length; i++) {
            result = lcm(result, input[i]);
        }
 
        return result;
    }
    //BINARY TREE::
    public class Node{
        int data;
        Node left,right;
        Node(int data){
            this.data = data;
            left = right = null;
        }

        public void insert(int value){
            if(value<=data){
                if(left==null) left = new Node(value);
                else left.insert(value);
            }
            else{
                if(right==null) right = new Node(value);
                else right.insert(value);
            }
        }  

        public boolean contains(int value){
           if(value==data) return true;
            else if(value<data){
                if(left==null) return false;
                else return left.contains(value);
            }
            else{
                if(right==null) return false;
                else return right.contains(value);
            }
        }
    }
    //FOR FINDING HEIGHT OF A TREE::
    public int maxht(Node root){
        if(root == null) return 0;
        else{
            int ld = maxht(root.left);
            int rd = maxht(root.right);
            
            if(ld>rd) return ld+1;
            else return rd+1;
        }
    }
    //Graph::
    public class graph{
        ArrayList<Integer> a;
        int d;
        boolean b;

        void graph(){
            this.d = 0;
            this.b = false;
        }
    }
    //Code starts right here... 
    public void run()
    {
        InputReader sc = new InputReader(System.in);
        PrintWriter w = new PrintWriter(System.out);
        /*write your code here*/
        /*take input through sc.nextInt and all and output through w.print & w.println*/
        int a[] = new int[(int)1e6+1];
        for(int c=2;c<(int)(1e6/2);c++){
            for(int j=c*2;j<=(int)1e6;j+=c){
                a[j]++;
            }
        }
        int n = sc.nextInt();
        long b[] = new long[n];
        for(int c=0;c<n;c++){
            long x = sc.nextLong();
            long y = sc.nextLong();
            b[c] = lcm(x,y);
        }
        long gcd = gcd(b);
        //w.println(gcd);
        for(int c=2;c<=(int)gcd;c++){
            if(a[c]==0 && gcd%c==0){w.println(c); break;}
        }
        if(gcd==1) w.println(-1);
        
        w.close();
    }
    
}
</code2>
Reply with only Yes or No. 
"""

# 编码并生成
with torch.inference_mode():
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 打印结果
print("\n===============================")
print("Prompt:")
print(prompt.strip())
print("===============================")
print("Model Output:")
print(result.strip())
print("===============================")

In [ ]:
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

# ========== Configuration ==========
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = None
TRUNCATION = False if MAX_LENGTH is None else True
# ===================================

# ---------- Remove comments ----------
def remove_comments(code: str) -> str:
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    code = re.sub(r"//.*", "", code)
    code = re.sub(r"#.*", "", code)
    code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
    code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
    return code.strip()

# ---------- Load model ----------
print(f"[Info] Loading model {model_name} ...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

try:
    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(load_in_8bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cuda",
        quantization_config=bnb_config,
    )
    print("[Info] Model loaded in 8-bit on CUDA.")
except Exception as e:
    print(f"[Warn] 8-bit load failed ({e}), falling back to fp16...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="cuda" if device == "cuda" else None,
    )

# ---------- Define helper ----------
def first_token_id(s: str):
    ids = tokenizer(s, add_special_tokens=False).input_ids
    return ids[0] if len(ids) > 0 else None

YES_CANDIDATES = [" Yes", "Yes", " yes"]
NO_CANDIDATES  = [" No", "No", " no"]

yes_ids = [tid for s in YES_CANDIDATES for tid in [first_token_id(s)] if tid is not None]
no_ids  = [tid for s in NO_CANDIDATES  for tid in [first_token_id(s)] if tid is not None]

def infer_single(prompt: str):
    try:
        inputs = tokenizer(
            prompt, return_tensors="pt",
            truncation=TRUNCATION, max_length=MAX_LENGTH
        )
        if device == "cuda":
            inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = model(**inputs)
        logits = outputs.logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        yes_prob = probs[:, yes_ids].sum().item()
        no_prob  = probs[:,  no_ids].sum().item()
        return "Yes" if yes_prob > no_prob else "No"

    except torch.cuda.OutOfMemoryError:
        torch.cuda.empty_cache()
        return "Error: CUDA OOM"
    except Exception as e:
        return f"Error: {e}"

# ---------- Example prompt ----------
problem = "Check whether these two implementations solve the same task."
code1 = """def add(a, b): return a + b"""
code2 = """def add(x, y): return x + y"""

prompt = (
    "Determine if the two code snippets are functionally equivalent according to the problem.\n\n"
    f"<problem>\n{problem}\n</problem>\n"
    f"<code1>\n{remove_comments(code1)}\n</code1>\n"
    f"<code2>\n{remove_comments(code2)}\n</code2>\n"
    "Reply with only Yes or No."
)

# ---------- Run ----------
result = infer_single(prompt)
print("\n==========================")
print("Prompt:\n", prompt)
print("==========================")
print("Model Answer:", result)


# Qwen with reason

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
import re
import gc

# ========== User Configuration ==========
input_file = "filtered_output.xlsx"
startline = 1
endline = 3
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
output_file = f"Qwen/full_equiv_output3_NoMismatch_{startline}_{endline}_YesNoReason_Qwen.xlsx"
device = "cuda" if torch.cuda.is_available() else "cpu"
# ========================================

# --- Helper: Remove comments ---
def remove_comments(code: str) -> str:
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    code = re.sub(r"//.*", "", code)
    code = re.sub(r"#.*", "", code)
    code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
    code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
    return code.strip()

# --- Load model (memory efficient) ---
print(f"Loading model {model_name} ...")

bnb_config = BitsAndBytesConfig(load_in_8bit=True)  # Optional: reduce VRAM use
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,     # comment out if you want full precision
    low_cpu_mem_usage=True,
)
model.eval()
print("Model loaded successfully.")

# --- Load Excel data ---
df = pd.read_excel(input_file)

# --- Define code pairs for comparison ---
pairs = [
    ("CPP_Pass1", "CPP_Pass2"),
    ("JAVA_Pass1", "JAVA_Pass2"),
    ("PYTHON3_Pass1", "PYTHON3_Pass2"),
    ("CPP_Pass1", "CPP_Fail"),
    ("JAVA_Pass1", "JAVA_Fail"),
    ("PYTHON3_Pass1", "PYTHON3_Fail"),
    ("CPP_Pass1", "JAVA_Pass1"),
    ("JAVA_Pass1", "PYTHON3_Pass1"),
    ("PYTHON3_Pass1", "CPP_Pass1"),
]

# --- Create empty result columns ---
for col1, col2 in pairs:
    df[f"{col1}_vs_{col2}"] = ""          # Yes/No
    df[f"{col1}_vs_{col2}_Reason"] = ""   # Explanation

# --- Process each row ---
for idx in range(startline - 1, min(endline, len(df))):
    print(f"Processing row {idx + 1}...")
    row = df.iloc[idx]
    problem = str(row.get("Description", "")).strip()

    for col1, col2 in pairs:
        code1 = str(row.get(col1, "")).strip()
        code2 = str(row.get(col2, "")).strip()
        if not code1 or not code2:
            continue

        clean_code1 = remove_comments(code1)
        clean_code2 = remove_comments(code2)

        prompt = f"""
You are a senior software analysis expert.

Determine whether the following two code snippets (<code1> and <code2>)
that aim to solve the same problem (<Problem>) are functionally equivalent.
Functional equivalence means they produce the same outputs for identical inputs.

1. First, answer with "Yes" or "No" on whether they are functionally equivalent.
2. Then, give a one-sentence explanation why you reached that conclusion.

Format your answer like this:
Answer: Yes
Reason: They both implement the same sorting logic but with different syntax.

<Problem>
{problem}
</Problem>

<code1>
{clean_code1}
</code1>

<code2>
{clean_code2}
</code2>
"""

        try:
            with torch.no_grad():
                inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(device)
                outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.0, do_sample=False)
                answer_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

            # Parse result
            ans_line = ""
            reason_line = ""

            # Try to extract "Answer:" and "Reason:"
            match_answer = re.search(r"(?i)answer\s*:\s*(yes|no)", answer_text)
            if match_answer:
                ans_line = match_answer.group(1).capitalize()

            match_reason = re.search(r"(?i)reason\s*:\s*(.*)", answer_text)
            if match_reason:
                reason_line = match_reason.group(1).strip()

            if not ans_line:
                ans_line = "Unclear"
            if not reason_line:
                reason_line = answer_text  # fallback if model didn’t format cleanly

            df.at[idx, f"{col1}_vs_{col2}"] = ans_line
            df.at[idx, f"{col1}_vs_{col2}_Reason"] = reason_line

        except torch.cuda.OutOfMemoryError:
            print("⚠️ CUDA out of memory — clearing cache and continuing...")
            torch.cuda.empty_cache()
            df.at[idx, f"{col1}_vs_{col2}"] = "Error: CUDA OOM"
            df.at[idx, f"{col1}_vs_{col2}_Reason"] = ""
        except Exception as e:
            df.at[idx, f"{col1}_vs_{col2}"] = "Error"
            df.at[idx, f"{col1}_vs_{col2}_Reason"] = str(e)

        # ✅ Free GPU memory each iteration
        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()

# --- Save the results ---
df.to_excel(output_file, index=False)
print(f"✅ Results saved to {output_file}")
